## Results: Masking strategies

In [1]:
import sys
sys.path.append("C:/Users/matth/Documents/Martinos Center/mrtoct") 

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cm
import utils.test_utils as utils
import os
import pickle
import numpy as np
import gc

In [2]:
path_model = "C:/Users/matth/Documents/Martinos Center/Models/DeepBrain/"
path_dataset = "C:/Users/matth/Documents/Martinos Center/mrtoct/datasets/"

### Python commands

Run these lines in the command prompt to generate the dataset and train models.

##### Preprocessing:

* <font color = blue>python preprocessing.py --dataset all_mask_sqrt</font>
* <font color = blue>python preprocessing.py --dataset all_nomask_sqrt --no_mask</font>
* <font color = blue>python preprocessing.py --dataset ct_mask_sqrt --mask_opt 1</font>
* <font color = blue>python preprocessing.py --dataset t1_mask_sqrt --mask_opt 2</font>

##### Training: 
        
* <font color = blue>python train.py --dataset all_nomask_sqrt --name nomask_nobim_30epochs --n_epochs 30</font>
* <font color = blue>python train.py --dataset all_nomask_sqrt --name nomask_30epochs --n_epochs 30 3</font>
* <font color = blue>python train.py --dataset ct_mask_sqrt --name ct_mask_nobim --no_bim --n_epochs 30</font>
* <font color = blue>python train.py --dataset ct_mask_sqrt --name ct_mask --n_epochs 30</font>
* <font color = blue>python train.py --dataset t1_mask_sqrt --name t1_mask_nobim --no_bim --n_epochs 30</font>
* <font color = blue>python train.py --dataset t1_mask_sqrt --name t1_mask --n_epochs 30</font>
* <font color = blue>python train.py --dataset all_mask_sqrt --name nobim_30epochs --no_bim --n_epochs 30</font>
* <font color = blue>python train.py --dataset all_mask_sqrt --name default_30epochs --n_epochs 30</font>

##### Retrieve data

In [ ]:
model_names = ["default_30epochs", "nobim_30epochs", "nomask_30epochs", "nomask_nobim_30epochs", "3d_60epochs", 
               "3d_l1_l2_nobim_60epochs"]

models, outputs = utils.retrieve_models(model_names, path_model)
trains, vals, bevels, begens, begts = utils.retrieve_history(outputs)

##### Plot

In [ ]:
fig, ax = plt.subplots(figsize = (16,8))
x = np.linspace(0, 30, len(trains[0]))

ax.plot(x, vals[0], 'r', label = 'With Mask/With BIM (Default)', linewidth = 2)#, marker = 'o', mfc = 'none')
ax.plot(x, vals[1], 'b', label = 'With Mask/No BIM', linewidth = 2)#, marker = 'o', mfc = 'none')
ax.plot(x, vals[2], 'g', label = 'No Mask/With BIM', linewidth = 2)#, marker = 'o', mfc = 'none')
ax.plot(x, vals[3], 'k', label = 'No Mask/No BIM', linewidth = 2)#, marker = 'o', mfc = 'none')

ax.set_title("Masking of non-skull pixels in the input data\nand during backpropagation: validation curves", fontsize = 25)
ax.set_xlabel("Epochs", fontsize = 22)
ax.set_ylabel("MAE", fontsize = 22)

ax.tick_params(axis='both', which='major', labelsize=25)
ax.set_ylim([0.08, 0.400])
ax.set_xlim([0, 30])
ax.legend(fontsize = 20, loc = 'upper right')
ax.grid('on')

In [ ]:
rows = 2
suptitles = ["2D UNet (256x256)", "3D ResNet (32x32x32)"]
fig, ax = plt.subplots(rows, 1, figsize = (12,10))
x = np.linspace(0, 30, len(trains[0]))
x2 = np.linspace(0, 60, len(trains[5]))

ax[0].plot(x, vals[0], 'r', label = 'With BIM', linewidth = 3)
ax[0].plot(x, vals[1], 'b', label = 'No BIM', linewidth = 3)

ax[1].plot(x2, vals[4], 'r', label = 'With BIM', linewidth = 3)
ax[1].plot(x2, vals[5], 'b', label = 'No BIM', linewidth = 3)

xlims = [[0,30], [0,60]]
ylims = [[0.05, 0.280], [0.06, 0.200]]

for i in range(rows):
    ax[i].set_xlabel("Epochs", fontsize = 22)
    ax[i].set_ylabel("Mean Absolute Error\n(MAE)", fontsize = 22)
    ax[i].tick_params(axis='both', which='major', labelsize=18)
    ax[i].set_ylim(ylims[i])
    ax[i].set_xlim(xlims[i])
    ax[i].legend(fontsize = 20, loc = 'upper right')    
    ax[i].grid("on")
    ax[i].set_title(suptitles[i], fontsize=25)
    
fig.subplots_adjust(right=1, wspace=0, hspace=0.5)

##### Metrics (2D)

In [ ]:
del models
gc.collect()

model_names = ["nomask_nobim_30epochs", "nomask_30epochs", "ct_mask_nobim", "ct_mask", "t1_mask_nobim", 
               "t1_mask", "nobim_30epochs",  "default_30epochs"]

models, outputs = utils.retrieve_models(model_names, path_model)
trains, vals, bevels, begens, begts = utils.retrieve_history(outputs)

In [ ]:
dataset_nomask = np.load(path_dataset + "all_nomask_sqrt" + "/valid_eval.npz")
dataset_ct = np.load(path_dataset + "ct_mask_sqrt" + "/valid_eval.npz")
dataset_t1 = np.load(path_dataset + "t1_mask_sqrt" + "/valid_eval.npz")
dataset_mask = np.load(path_dataset + "all_mask_sqrt" + "/valid_eval.npz")
datasets = [dataset_nomask, dataset_nomask, dataset_ct, dataset_ct, dataset_t1, dataset_t1, dataset_mask, dataset_mask]

metrics = utils.compute_val_metrics(models, model_names, datasets, bevels, begens, begts)
metrics

In [3]:
model_names = ["ct_mask", "ct_mask_tanh_30ep"]

models, outputs = utils.retrieve_models(model_names, path_model)
trains, vals, bevels, begens, begts = utils.retrieve_history(outputs)

In [4]:
dataset_sigmoid = np.load(path_dataset + "ct_mask_sqrt" + "/valid_eval.npz")
dataset_tanh = np.load(path_dataset + "ct_mask_sqrt_tanh" + "/valid_eval.npz")

datasets = [dataset_sigmoid, dataset_tanh]
metrics = utils.compute_val_metrics(models, model_names, datasets, bevels, begens, begts)
metrics

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:20<00:00, 40.36s/it]


,MAE,MSE,NRMSE,PSNR,SSIM
,,,,,
ct_mask,0.0954,0.0349,0.1868,33.5552,0.9754
ct_mask_tanh_30ep,0.0958,0.0353,0.1880,47.2912,0.9623


In [ ]:
source = dataset_sigmoid['x'][0:1, :, :, :]
target = dataset_sigmoid['y'][0, :, :, 0]
mask = dataset_sigmoid['m'][0, :, :, 0]

fig, ax = plt.subplots(1,3, figsize = (15,10))
for i in range(2):
    if i == 1:
        source = ( source - 0.5 ) * 2
    print(source.shape)
    pred = models[i].predict(source)
    print(pred.shape)
    ax[i].imshow( ( pred[0,:,:,0] - 1 ) * mask + 1 , cmap = "gray")
    
ax[2].imshow(target, cmap = "gray")

In [9]:
with open(path_dataset + "all_mask_sqrt" + "/dataset_info", "rb") as f:
    data = pickle.load(f)
#dataset_sigmoid = pickle.load(path_dataset + "ct_mask_sqrt" + "dataset_info")
#print(data["elapsed time"])
print(data)

{'mm_t1': MinMaxScaler(), 'mm_ct': MinMaxScaler(), 'mm_mask': MinMaxScaler(), 'no_mask': False, 'no_sqrt': False, 'seed': 0, 'set_pct': False, 'ns_valid': 473, 'patch_shape': (256, 256, 1)}
